In [3]:
from langchain.llms import GooglePalm
import os

api_key = os.environ.get('GOOGLE_API_KEY')

llm = GooglePalm(temperature=0.7)

In [4]:
result = llm('Write a short story about nepal')
print(result)

The sun was shining brightly in the blue sky as the birds chirped happily in the trees. A gentle breeze blew through the air, rustling the leaves. It was a beautiful day in Nepal.

A young girl named Anya was walking through the forest. She was on her way to visit her grandmother, who lived on the other side of the mountain. Anya was excited to see her grandmother, and she couldn't wait to tell her all about her day.

As Anya walked, she noticed a beautiful flower. She had never seen a flower like it before. It was a deep purple color, and it had a sweet scent. Anya picked the flower and carefully placed it in her hair.

Anya continued on her way, and soon she came to a clearing. In the middle of the clearing was a large tree. Anya sat down under the tree and began to eat her lunch. As she ate, she thought about her grandmother. She wondered what she would be doing right now. Would she be baking bread? Or would she be sitting in her garden, tending to her flowers?

After Anya finished 

In [ ]:
text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities. Answer in lines"
print(llm(text))

# The chains

In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)


chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("eco-friendly water bottles"))

**BottleDrop**


# Chaining multiple chains

In [6]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SimpleSequentialChain

In [9]:
# This is an LLMChain to write first chain.
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)
chain_one = LLMChain(llm=llm, prompt=first_prompt)

# This is an LLMChain to write second chain.
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company:{company_name}"
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [10]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )
overall_simple_chain.run("gaming laptop")



> Entering new  chain...


Retrying langchain.llms.google_palm.generate_with_retry.<locals>._generate_with_retry in 2.0 seconds as it raised ServiceUnavailable: 503 Connection timed out.


Alienware
designs and builds high-performance personal computers and related accessories. Alienware

> Finished chain.


'designs and builds high-performance personal computers and related accessories. Alienware'

# The memory

In [17]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=memory
)

# Start the conversation
conversation.predict(input="Who won fifa 2018")




> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Who won fifa 2018
AI:

> Finished chain.


'France'

In [18]:
# Continue the conversation
conversation.predict(input="what is 2+2")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Who won fifa 2018
AI: France
Human: what is 2+2
AI:

> Finished chain.


'4'

In [19]:
conversation.predict(input="Who was the captain of winning team")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Who won fifa 2018
AI: France
Human: what is 2+2
AI: 4
Human: Who was the captain of winning team
AI:

> Finished chain.


'Hugo Lloris'

In [20]:
print(conversation.memory.buffer)

Human: Who won fifa 2018
AI: France
Human: what is 2+2
AI: 4
Human: Who was the captain of winning team
AI: Hugo Lloris


## Limiting the memory limit

In [26]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)

conversation = ConversationChain(
    llm=GooglePalm(temperature=0),
    verbose=True,
    memory=memory
)

In [27]:
# Start the conversation
conversation.predict(input="Who won fifa 2018")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Who won fifa 2018
AI:

> Finished chain.


'France'

In [28]:
# Continue the conversation
conversation.predict(input="what is 2+2")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Who won fifa 2018
AI: France
Human: what is 2+2
AI:

> Finished chain.


'4'

In [29]:
conversation.predict(input="Who was the captain of winning team")



> Entering new  chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: what is 2+2
AI: 4
Human: Who was the captain of winning team
AI:

> Finished chain.


'Lionel Messi'

# Deep Lake VectorStore

In [1]:
from langchain.embeddings.google_palm import GooglePalmEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import GooglePalm
from langchain.chains import RetrievalQA

In [2]:
# instantiate the LLM and embeddings models
llm = GooglePalm(temperature=0)
embeddings = GooglePalmEmbeddings()

c:\Users\dell\OneDrive\Documents\LangChain and Vector database\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

In [5]:
my_activeloop_org_id = "samman" 
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

Your Deep Lake dataset has been successfully created!


In [6]:
# add documents to our Deep Lake dataset
db.add_documents(docs)

Creating 2 embeddings in 1 batches of size 2:: 100%|██████████| 1/1 [00:24<00:00, 24.38s/it]

Dataset(path='hub://samman/langchain_course_from_zero_to_hero', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype     shape     dtype  compression
  -------    -------   -------   -------  ------- 
   text       text      (2, 1)     str     None   
 metadata     json      (2, 1)     str     None   
 embedding  embedding  (2, 768)  float32   None   
    id        text      (2, 1)     str     None   


['92be1184-9b23-11ee-b60b-60189524c791',
 '92be1185-9b23-11ee-98a7-60189524c791']

In [7]:
# Creating a retrieval QA chain
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

In [8]:
# create an agent that uses the RetrievalQA chain as a tool
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [9]:
# use the agent to ask a question
response = agent.run("When was Napoleone born?")
print(response)



> Entering new  chain...
I need to find out when Napoleone was born
Action: Retrieval QA System
Action Input: when was Napoleone born
Observation: 15 August 1769
Thought:I now know the final answer
Final Answer: 15 August 1769

> Finished chain.
15 August 1769


In [10]:
# load the existing Deep Lake dataset and specify the embedding function
db = DeepLake(dataset_path=dataset_path, embedding_function=embeddings)

# create new documents
texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

# add documents to our Deep Lake dataset
db.add_documents(docs)

Deep Lake Dataset in hub://samman/langchain_course_from_zero_to_hero already exists, loading from the storage


Creating 2 embeddings in 1 batches of size 2:: 100%|██████████| 1/1 [00:35<00:00, 35.74s/it]

Dataset(path='hub://samman/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype     shape     dtype  compression
  -------    -------   -------   -------  ------- 
 embedding  embedding  (4, 768)  float32   None   
    id        text      (4, 1)     str     None   
 metadata     json      (4, 1)     str     None   
   text       text      (4, 1)     str     None   


['73f24f68-9b24-11ee-b082-60189524c791',
 '73f24f69-9b24-11ee-b8f9-60189524c791']

In [11]:
# create a retriever from the db
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm, chain_type="stuff", retriever=db.as_retriever()
)

# instantiate a tool that uses the retriever
tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.run,
        description="Useful for answering questions."
    ),
]

# create an agent that uses the tool
agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [12]:
response = agent.run("When was Michael Jordan born?")
print(response)



> Entering new  chain...
I need to find out when Michael Jordan was born
Action: Retrieval QA System
Action Input: when was michael jordan born
Observation: 17 February 1963
Thought:I now know the final answer
Final Answer: 17 February 1963

> Finished chain.
17 February 1963
